In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
df_cost = pd.read_csv('../data/raw_data/costs_us.csv')
df_orders = pd.read_csv('../data/raw_data/orders_log_us.csv')
df_visits = pd.read_csv('../data/raw_data/visits_log_us.csv')


### Descrição dos Dados

O projeto utiliza três fontes de dados para analisar a performance de marketing:

*   **Tabela `visits` (Logs de Acesso):**
    *   `Uid`: ID do usuário
    *   `Device`: Dispositivo (desktop/touch)
    *   `Start Ts`: Início da sessão (data/hora)
    *   `End Ts`: Fim da sessão (data/hora)
    *   `Source Id`: ID da fonte de tráfego

*   **Tabela `orders` (Dados de Pedidos):**
    *   `Uid`: ID do usuário
    *   `Buy Ts`: Data e hora do pedido
    *   `Revenue`: Receita do pedido

*   **Tabela `costs` (Custos de Marketing):**
    *   `source_id`: ID da fonte de anúncio
    *   `dt`: Data do gasto
    *   `costs`: Valor gasto

In [16]:
df_cost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   source_id  2542 non-null   int64  
 1   dt         2542 non-null   object 
 2   costs      2542 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 59.7+ KB


In [17]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50415 entries, 0 to 50414
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Buy Ts   50415 non-null  object 
 1   Revenue  50415 non-null  float64
 2   Uid      50415 non-null  uint64 
dtypes: float64(1), object(1), uint64(1)
memory usage: 1.2+ MB


In [18]:
df_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359400 entries, 0 to 359399
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Device     359400 non-null  object
 1   End Ts     359400 non-null  object
 2   Source Id  359400 non-null  int64 
 3   Start Ts   359400 non-null  object
 4   Uid        359400 non-null  uint64
dtypes: int64(1), object(3), uint64(1)
memory usage: 13.7+ MB


In [19]:
# Renomeando as colunas
df_cost.rename(columns={'source_id': 'id', 'dt': 'date', 'costs': 'cost'}, inplace=True)
df_orders.rename(columns={'Uid': 'uid', 'Buy Ts': 'buy_date', 'Revenue': 'revenue'}, inplace=True)
df_visits.rename(columns={'Uid': 'uid', 'Device': 'device', 'Start Ts': 'start_date', 'End Ts': 'end_date', 'Source Id': 'id'}, inplace=True)

In [20]:
# Adequação dos tipos 

# df_cost
# Convertendo a coluna 'date' para datetime
df_cost['date'] = pd.to_datetime(df_cost['date'])

# df_orders
# Convertendo a coluna 'buy_date' para datetime
df_orders['buy_date'] = pd.to_datetime(df_orders['buy_date'])

# df_visits
# Convertendo as colunas de data para datetime
df_visits['start_date'] = pd.to_datetime(df_visits['start_date'])
df_visits['end_date'] = pd.to_datetime(df_visits['end_date'])

# Convertendo a coluna 'device' para category
df_visits['device'] = df_visits['device'].astype('category')

In [21]:
# Verificação de Nulos 
print("Valores Nulos em df_cost:")
print(df_cost.isnull().sum())
print("\nValores Nulos em df_orders:")
print(df_orders.isnull().sum())
print("\nValores Nulos em df_visits:")
print(df_visits.isnull().sum())


Valores Nulos em df_cost:
id      0
date    0
cost    0
dtype: int64

Valores Nulos em df_orders:
buy_date    0
revenue     0
uid         0
dtype: int64

Valores Nulos em df_visits:
device        0
end_date      0
id            0
start_date    0
uid           0
dtype: int64



### Processamento e Limpeza dos Dados

Nesta fase inicial, os dados brutos foram preparados para a análise, garantindo sua qualidade e consistência. O processo foi dividido em três etapas principais:

**1. Carga dos Dados:**
*   Foram carregados três arquivos CSV: `visits_log_us.csv`, `orders_log_us.csv` e `costs_us.csv`, utilizando a biblioteca Pandas.

**2. Limpeza e Adequação:**
*   **Renomeação das Colunas:** Os nomes das colunas foram padronizados para facilitar o acesso e a manipulação (ex: `Uid` para `uid`, `Start Ts` para `start_date`).
*   **Ajuste de Tipos:** Todas as colunas contendo datas e horas foram convertidas para o formato `datetime`, permitindo análises temporais precisas.
*   **Otimização:** A coluna `device` foi convertida para o tipo `category` para otimizar o uso de memória.

**3. Validação dos Dados:**
*   **Verificação de Nulos e Duplicatas:** Foi realizada uma checagem final para confirmar a ausência de valores nulos ou linhas duplicadas em todos os conjuntos de dados. O resultado confirmou que os dados estão completos e íntegros.


In [24]:
# --- Passo 1: Encontrar a primeira visita (Coorte de Aquisição) ---
# Agrupa as visitas por usuário e pega a data da primeira sessão
first_visits = df_visits.groupby('uid')['start_date'].min().reset_index()
first_visits.columns = ['uid', 'acquisition_date']

# Cria a coluna de coorte (formato YYYY-MM)
first_visits['acquisition_cohort'] = first_visits['acquisition_date'].dt.to_period('M')


# --- Passo 2: Juntar tudo em uma tabela final ---
# Junta a coorte de aquisição com a tabela de visitas
df_merged = pd.merge(df_visits, first_visits, on='uid')

# Junta o resultado com a tabela de pedidos
# Usamos 'left' para manter todas as visitas, mesmo as que não tiveram pedidos
df_final = pd.merge(df_merged, df_orders, on='uid', how='left')


# --- Passo 3: Criar colunas de data para análise ---
# Cria a coluna do mês da atividade (visita ou pedido)
df_final['activity_month'] = df_final['start_date'].dt.to_period('M')
df_final['order_month'] = df_final['buy_date'].dt.to_period('M')


# --- Passo 4: Juntar os custos ---
# Agrega os custos por mês
monthly_costs = df_cost.groupby(df_cost['date'].dt.to_period('M'))['cost'].sum().reset_index()
monthly_costs.columns = ['activity_month', 'monthly_total_cost']

# Junta os custos mensais à tabela final
df_final = pd.merge(df_final, monthly_costs, on='activity_month', how='left')



# Exibe as primeiras linhas da tabela final
df_final.head()

,device,end_date,id,start_date,uid,acquisition_date,acquisition_cohort,buy_date,revenue,activity_month,order_month,monthly_total_cost
0,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00,16879256277535980062,2017-12-20 17:20:00,2017-12,NaT,NaN,2017-12,NaT,38315.35
1,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00,104060357244891740,2018-02-19 16:53:00,2018-02,NaT,NaN,2018-02,NaT,32723.03
2,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00,7459035603376831527,2017-07-01 01:54:00,2017-07,NaT,NaN,2017-07,NaT,18240.59
3,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00,16174680259334210214,2018-03-09 20:05:00,2018-03,2018-03-09 20:25:00,2.33,2018-05,2018-03,22224.27
4,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00,9969694820036681168,2017-12-27 14:06:00,2017-12,NaT,NaN,2017-12,NaT,38315.35
